In [32]:
import pandas as pd

# Load the csv data as a pandas dataframe
google_stock_df = pd.read_csv('datasets/GOOGL.csv')

open_price_data = google_stock_df['Open']
# normalized_open_data = (open_price_data-open_price_data.min())/(open_price_data.max()-open_price_data.min())
normalized_open_data = open_price_data
# Separate into train a test data
train_data = normalized_open_data[:-1000]
test_data = normalized_open_data[-1000:]



In [61]:
from SMRNN import SMRNN
from torch import optim
import torch.nn as nn
# Create the network optimizer and loss function
net = SMRNN(inputs = 1, outputs = 1, inner_state_size = 5,net1_inner_topology = [10,10,20,10,5,2], net2_inner_topology = [7,15,30,70],net3_inner_topology = [7,7,5])
optimizer = optim.Adam(net.parameters(),lr=1e-2)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer,0.8)
loss_func = nn.MSELoss()



In [16]:
import torch
#detect gpu and move the net there
torch.cuda.is_available()

False

In [62]:
import random
from matplotlib import pyplot
# train the network (in this case only with the open price)
epochs = 3000
# the batch sizes will be random to let the model to learn in diferent lengths
max_batch_size = 30
min_batch_size = 10
losses = []
for epoch in range(epochs):
    batch_size = random.randint(5,20)
    train_index = random.randint(0,train_data.size - max_batch_size - 2)
    for batch_num in range(batch_size):
        inpt = torch.tensor([float(train_data[train_index+batch_num])])
        output = net(inpt)
    expected_out = torch.tensor([float(train_data[train_index+batch_size])])
    loss = torch.abs(output - expected_out)
    # loss = loss_func(output,expected_out)
    print(loss)
    print(f'Predicted: {float(output)}, actual: {float(expected_out)}')
    losses.append(float(loss))
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if epoch % 100 == 99:
        scheduler.step()
    net.clean()

pyplot.plot(list(range(epochs)),losses)
pyplot.show()
    

tensor([561.4161], grad_fn=<AbsBackward0>)
Predicted: -0.5160898566246033, actual: 560.9000244140625
tensor([576.7125], grad_fn=<AbsBackward0>)
Predicted: -0.5713978409767151, actual: 576.14111328125
tensor([154.2886], grad_fn=<AbsBackward0>)
Predicted: -0.5248371958732605, actual: 153.76376342773438
tensor([345.8461], grad_fn=<AbsBackward0>)
Predicted: -0.5007944107055664, actual: 345.3453369140625
tensor([546.2161], grad_fn=<AbsBackward0>)
Predicted: -0.4760899245738983, actual: 545.739990234375
tensor([712.3860], grad_fn=<AbsBackward0>)
Predicted: -0.4660898745059967, actual: 711.9199829101562
tensor([777.7661], grad_fn=<AbsBackward0>)
Predicted: -0.4560898244380951, actual: 777.3099975585938
tensor([99.0293], grad_fn=<AbsBackward0>)
Predicted: -0.45070216059684753, actual: 98.57858276367188
tensor([275.3060], grad_fn=<AbsBackward0>)
Predicted: -0.4360896646976471, actual: 274.869873046875
tensor([290.8115], grad_fn=<AbsBackward0>)
Predicted: -0.4260895848274231, actual: 290.3853759

KeyboardInterrupt: 